# Imports

In [1]:
import os

import datetime

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Binarizer, OneHotEncoder, OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier

In [2]:
import sklearn
from sklearn import set_config

In [3]:
sklearn.__version__

'1.2.0'

# Global Configuration

In [4]:
set_config(transform_output="pandas")

In [76]:
CWD = os.getcwd()

In [77]:
CWD

'/Users/nicolaepopescul/Desktop/streams/20240328_sklearn_modelo_benchmark_ml'

In [82]:
PATH_INPUT = os.path.join(CWD, "input")

In [83]:
PATH_INPUT

'/Users/nicolaepopescul/Desktop/streams/20240328_sklearn_modelo_benchmark_ml/input'

In [5]:
PATH_INPUT_TRAIN = os.path.join(PATH_INPUT, "train.csv")
PATH_INPUT_TEST = os.path.join(PATH_INPUT, "test.csv")

In [6]:
os.listdir(CWD)

['submission_2024_03_30_19_20_52.csv',
 '.DS_Store',
 'requirements.txt',
 'Benchmark_20240328.ipynb',
 'input',
 'venvs',
 'Benchmark_20240329.ipynb',
 'Benchmark_20240330.ipynb',
 'submission_2024_03_30_19_35_18.csv',
 'submission_2024_03_30_19_25_22.csv',
 '2104203884-734476011-3058cb37-b96a-458a-9087-5d1085783958.mp4',
 '.ipynb_checkpoints']

In [7]:
os.listdir(PATH_INPUT)

['test.csv', 'spaceship-titanic.zip', 'train.csv', 'sample_submission.csv']

# Helper functions

In [8]:
def load_data():
    
    X_train = pd.read_csv(PATH_INPUT_TRAIN).set_index("PassengerId")
    X_test = pd.read_csv(PATH_INPUT_TEST).set_index("PassengerId")
    
    return X_train, X_test

# Dataset loading & EDA

In [9]:
X, _ = load_data()

In [10]:
(
    X
    .groupby("HomePlanet")
    ["Transported"]
    .mean()
    .sort_values(ascending = False)
    .plot(kind = "bar")
);

In [11]:
X.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [12]:
_ = (
    X
    .groupby(["HomePlanet", "CryoSleep"])
    .agg(
        nr_rows = ("Cabin", len),
        transported_ratio = ("Transported", np.mean)
    )
    .assign(
        nr_rows_pct = lambda df: round((df["nr_rows"]/df["nr_rows"].sum())*100, 2),
        transported_ratio = lambda df: round(df["transported_ratio"] * 100, 2)
    )
)

/var/folders/m8/pkh7yz5j3_318g64strn1y680000gn/T/ipykernel_2543/1725775422.py:2: FutureWarning: The provided callable <function mean at 0x111d1e550> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  X


In [13]:
_

nr_rows  transported_ratio  nr_rows_pct
HomePlanet CryoSleep                                         
Earth      False         3106              32.10        37.53
           True          1382              65.63        16.70
Europa     False         1162              40.02        14.04
           True           911              98.90        11.01
Mars       False         1047              27.70        12.65
           True           669              91.18         8.08

In [14]:
X["LenCabin"] = X["Cabin"].apply(lambda cabin: len(str(cabin)))

In [15]:
X.groupby(["LenCabin"]).size()

LenCabin
3     199
5     130
6    1201
7    5043
8    2120
dtype: int64

In [16]:
X[X["LenCabin"] == 5].head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,LenCabin
PassengerId,,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,5
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,5
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,5
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,5
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,5


In [17]:
X["Cabin"].fillna("NA", inplace = True)

/var/folders/m8/pkh7yz5j3_318g64strn1y680000gn/T/ipykernel_2543/3468940878.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X["Cabin"].fillna("NA", inplace = True)


In [18]:
X["CabinFirstLetter"] = X["Cabin"].apply(lambda cabin: cabin[0] if cabin != "NA" else cabin)
X["CabinLastLetter"] = X["Cabin"].apply(lambda cabin: cabin[-1] if cabin != "NA" else cabin)

In [19]:
X["CabinFirstLetter"].value_counts().sort_index()

CabinFirstLetter
A      256
B      779
C      747
D      478
E      876
F     2794
G     2559
NA     199
T        5
Name: count, dtype: int64

In [20]:
X["CabinLastLetter"].value_counts().sort_index()

CabinLastLetter
NA     199
P     4206
S     4288
Name: count, dtype: int64

In [21]:
X["Cabin"].sample(10)

PassengerId
1728_01     G/280/P
8845_01    F/1823/P
6774_02    G/1100/P
7644_01    G/1227/P
7038_01    G/1137/P
8978_01    F/1841/P
6443_01    G/1046/S
3260_01     F/615/S
2092_01      D/70/S
2628_02      A/21/P
Name: Cabin, dtype: object

In [22]:
(
    X
    .groupby(["CabinFirstLetter"])
    ["Transported"]
    .mean()
    .sort_index()
)

CabinFirstLetter
A     0.496094
B     0.734275
C     0.680054
D     0.433054
E     0.357306
F     0.439871
G     0.516217
NA    0.502513
T     0.200000
Name: Transported, dtype: float64

In [23]:
(
    X
    .groupby(["CabinLastLetter"])
    ["Transported"]
    .mean()
    .sort_index()
)

CabinLastLetter
NA    0.502513
P     0.451260
S     0.555037
Name: Transported, dtype: float64

In [24]:
pd.concat([
    (
        X
        .groupby(["CabinFirstLetter", "CabinLastLetter"])
        ["Transported"]
        .mean()
        .sort_index()
        .unstack()
    ),
    (
        X
        .groupby(["CabinFirstLetter"])
        ["Transported"]
        .mean()
        .sort_index()
    )], axis = 1)

,NA,P,S,Transported
CabinFirstLetter,,,,
A,NaN,0.435897,0.546763,0.496094
B,NaN,0.674221,0.784038,0.734275
C,NaN,0.580645,0.763547,0.680054
D,NaN,0.403226,0.465217,0.433054
E,NaN,0.342657,0.371365,0.357306
F,NaN,0.410987,0.470501,0.439871
G,NaN,0.448276,0.583788,0.516217
NA,0.502513,NaN,NaN,0.502513
T,NaN,0.250000,0.000000,0.200000


In [25]:
X.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,LenCabin
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000,8693.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791,6.984240
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189,0.898869
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000,7.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000,8.000000


In [26]:
X.isnull().sum()

HomePlanet          201
CryoSleep           217
Cabin                 0
Destination         182
Age                 179
VIP                 203
RoomService         181
FoodCourt           183
ShoppingMall        208
Spa                 183
VRDeck              188
Name                200
Transported           0
LenCabin              0
CabinFirstLetter      0
CabinLastLetter       0
dtype: int64

In [27]:
X.shape

(8693, 16)

In [28]:
(
    X
    .pivot_table(
        index = "HomePlanet",
        columns = "Destination",
        values = "Transported",
        aggfunc = [len, np.mean],
        margins = True
    )
)

/var/folders/m8/pkh7yz5j3_318g64strn1y680000gn/T/ipykernel_2543/4213924544.py:2: FutureWarning: The provided callable <function mean at 0x111d1e550> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  X
/var/folders/m8/pkh7yz5j3_318g64strn1y680000gn/T/ipykernel_2543/4213924544.py:2: FutureWarning: The provided callable <function mean at 0x111d1e550> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  X
/var/folders/m8/pkh7yz5j3_318g64strn1y680000gn/T/ipykernel_2543/4213924544.py:2: FutureWarning: The provided callable <function mean at 0x111d1e550> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  X


len                                        mean  \
Destination 55 Cancri e PSO J318.5-22 TRAPPIST-1e   All 55 Cancri e   
HomePlanet                                                            
Earth               690           712        3101  4503    0.504348   
Europa              886            19        1189  2094    0.689616   
Mars                193            49        1475  1717    0.611399   
All                1769           780        5765  8314    0.608819   

                                                 
Destination PSO J318.5-22 TRAPPIST-1e       All  
HomePlanet                                       
Earth            0.498596    0.389229  0.424162  
Europa           0.736842    0.634987  0.659026  
Mars             0.448980    0.511864  0.521258  
All              0.501282    0.471292  0.503368

In [29]:
X["Name"].fillna("NA").apply(lambda name: len(name.split(" "))).value_counts()

Name
2    8493
1     200
Name: count, dtype: int64

In [30]:
X["Surname"] = X["Name"].fillna("NA").apply(lambda name: name.split(" ")[1] if name != "NA" else name)

In [31]:
X.groupby(["Surname"])["Transported"].mean()

Surname
Acobson        0.250000
Acobsond       0.333333
Adavisons      0.444444
Adkinson       0.000000
Admingried     0.750000
                 ...   
Wynneyerson    0.500000
Yanton         0.285714
Yatters        0.500000
Yorkland       0.636364
Youngrayes     0.333333
Name: Transported, Length: 2218, dtype: float64

In [32]:
del X, _

# Preprocessing and Modelling

In [33]:
X_train, X_test = load_data()

In [34]:
y = X_train.pop("Transported")

In [35]:
X_train.select_dtypes(exclude=np.number).columns.tolist()

['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Name']

In [36]:
class CabinTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self, verbose=False):
        self.verbose = verbose
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        
        if self.verbose:
            print("Cabin Transformer")
            print(type(X))
        
        return X.iloc[:, 0].apply(lambda cabin: cabin[0] if cabin != "NA" else cabin).to_frame()

In [37]:
ct_impute = ColumnTransformer(
    transformers = [
        ("impute_numerical", SimpleImputer(strategy="mean"), make_column_selector(dtype_include=np.number)),
        ("impute_cabin", SimpleImputer(strategy="constant", fill_value="NA"), ["Cabin"]),
        (
            "impute_categorical",
            SimpleImputer(strategy="most_frequent"),
            ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Name']
        )
    ],
    remainder = 'drop'
)

In [38]:
ct_impute

ColumnTransformer(transformers=[('impute_numerical', SimpleImputer(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x166636e80>),
                                ('impute_cabin',
                                 SimpleImputer(fill_value='NA',
                                               strategy='constant'),
                                 ['Cabin']),
                                ('impute_categorical',
                                 SimpleImputer(strategy='most_frequent'),
                                 ['HomePlanet', 'CryoSleep', 'Destination',
                                  'VIP', 'Name'])])

In [39]:
Xt = ct_impute.fit_transform(X_train)

In [40]:
Xt.shape

(8693, 12)

In [41]:
Xt.head()

,impute_numerical__Age,impute_numerical__RoomService,impute_numerical__FoodCourt,impute_numerical__ShoppingMall,impute_numerical__Spa,impute_numerical__VRDeck,impute_cabin__Cabin,impute_categorical__HomePlanet,impute_categorical__CryoSleep,impute_categorical__Destination,impute_categorical__VIP,impute_categorical__Name
PassengerId,,,,,,,,,,,,
0001_01,39.0,0.0,0.0,0.0,0.0,0.0,B/0/P,Europa,False,TRAPPIST-1e,False,Maham Ofracculy
0002_01,24.0,109.0,9.0,25.0,549.0,44.0,F/0/S,Earth,False,TRAPPIST-1e,False,Juanna Vines
0003_01,58.0,43.0,3576.0,0.0,6715.0,49.0,A/0/S,Europa,False,TRAPPIST-1e,True,Altark Susent
0003_02,33.0,0.0,1283.0,371.0,3329.0,193.0,A/0/S,Europa,False,TRAPPIST-1e,False,Solam Susent
0004_01,16.0,303.0,70.0,151.0,565.0,2.0,F/1/S,Earth,False,TRAPPIST-1e,False,Willy Santantines


In [42]:
# CabinTransformer().fit_transform(Xt["impute_cabin__Cabin"])

In [43]:
CabinTransformer().fit_transform(Xt[["impute_cabin__Cabin"]])

,impute_cabin__Cabin
PassengerId,
0001_01,B
0002_01,F
0003_01,A
0003_02,A
0004_01,F
...,...
9276_01,A
9278_01,G
9279_01,G


In [44]:
# CabinTransformer().fit_transform(Xt[["impute_cabin__Cabin"]]).value_counts().sort_index()

In [45]:
CabinTransformer().transform(ct_impute.transform(X_test)[["impute_cabin__Cabin"]]).value_counts().sort_index()

impute_cabin__Cabin
A                        98
B                       362
C                       355
D                       242
E                       447
F                      1445
G                      1222
NA                      100
T                         6
Name: count, dtype: int64

In [46]:
fu_age = FeatureUnion(
    transformer_list = [
        ("fe_age_12", Binarizer(threshold=12)),
        ("fe_age_18", Binarizer(threshold=18)),
        ("fe_age_pass", "passthrough")       
    ]
)

In [47]:
Xt.columns.tolist()

['impute_numerical__Age',
 'impute_numerical__RoomService',
 'impute_numerical__FoodCourt',
 'impute_numerical__ShoppingMall',
 'impute_numerical__Spa',
 'impute_numerical__VRDeck',
 'impute_cabin__Cabin',
 'impute_categorical__HomePlanet',
 'impute_categorical__CryoSleep',
 'impute_categorical__Destination',
 'impute_categorical__VIP',
 'impute_categorical__Name']

In [48]:
# Xt["impute_cabin__Cabin"]

In [49]:
class DebuggerTransformer(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        print("DummyTransformer")
        return X

In [50]:
fe_cabin = Pipeline(
    steps = [
        ("extract_first_letter", CabinTransformer()),
        ("ohe_first_letter", OneHotEncoder(sparse_output=False)),
    ]
)

In [51]:
# Xt["impute_cabin__Cabin"]

In [52]:
fe_cabin[0].fit_transform(Xt[["impute_cabin__Cabin"]])

,impute_cabin__Cabin
PassengerId,
0001_01,B
0002_01,F
0003_01,A
0003_02,A
0004_01,F
...,...
9276_01,A
9278_01,G
9279_01,G


In [53]:
fe_cabin.fit_transform(Xt[["impute_cabin__Cabin"]])

,impute_cabin__Cabin_A,impute_cabin__Cabin_B,impute_cabin__Cabin_C,impute_cabin__Cabin_D,impute_cabin__Cabin_E,impute_cabin__Cabin_F,impute_cabin__Cabin_G,impute_cabin__Cabin_NA,impute_cabin__Cabin_T
PassengerId,,,,,,,,,
0001_01,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0002_01,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
0003_01,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0003_02,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0004_01,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
9276_01,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9278_01,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9279_01,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [54]:
# fe_cabin[1].fit_transform(fe_cabin[0].fit_transform(Xt["impute_cabin__Cabin"]).values.reshape(-1, 1))

("cabin_fe", make_column_transformer(
    (
        CabinTransformer(), ["impute_cabin__Cabin"])
    )
),

In [55]:
ct_fe = ColumnTransformer(
    transformers = [
        ("fu_age", fu_age, ["impute_numerical__Age"]),
        ("pass", "passthrough", [1, 2, 3, 4, 5]),
        ("fe_cabin", fe_cabin, ["impute_cabin__Cabin"]),
        (
            "ohe",
            OneHotEncoder(sparse_output=False),
            ["impute_categorical__HomePlanet", "impute_categorical__CryoSleep", "impute_categorical__VIP"]
        ),
        ("or", OrdinalEncoder(), ["impute_categorical__Destination"])
    ],
    remainder = "drop"
)

In [56]:
ct_fe

ColumnTransformer(transformers=[('fu_age',
                                 FeatureUnion(transformer_list=[('fe_age_12',
                                                                 Binarizer(threshold=12)),
                                                                ('fe_age_18',
                                                                 Binarizer(threshold=18)),
                                                                ('fe_age_pass',
                                                                 'passthrough')]),
                                 ['impute_numerical__Age']),
                                ('pass', 'passthrough', [1, 2, 3, 4, 5]),
                                ('fe_cabin',
                                 Pipeline(steps=[('extract_first_letter',
                                                  CabinTransformer()),
                                                 ('ohe_first_letter',
                                                  OneHotEncoder(sparse_output=False))]),
                                 ['impute_cabin__Cabin']),
                                ('ohe', OneHotEncoder(sparse_output=False),
                                 ['impute_categorical__HomePlanet',
                                  'impute_categorical__CryoSleep',
                                  'impute_categorical__VIP']),
                                ('or', OrdinalEncoder(),
                                 ['impute_categorical__Destination'])])

In [57]:
pipe = Pipeline(
    steps = [
        ("impute", ct_impute),
        ("fe", ct_fe),
        # ("model", DecisionTreeClassifier(max_depth=5))
        # ("model", RandomForestClassifier(n_estimators = 50, max_depth = 5))
        ("model",HistGradientBoostingClassifier())
    ]
)

In [58]:
pipe

Pipeline(steps=[('impute',
                 ColumnTransformer(transformers=[('impute_numerical',
                                                  SimpleImputer(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x166636e80>),
                                                 ('impute_cabin',
                                                  SimpleImputer(fill_value='NA',
                                                                strategy='constant'),
                                                  ['Cabin']),
                                                 ('impute_categorical',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['HomePlanet', 'CryoSleep',
                                                   'Destinatio...
                                                  Pipeline(steps=[('extract_first_letter',
                                                                   CabinTransformer()),
                                                                  ('ohe_first_letter',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  ['impute_cabin__Cabin']),
                                                 ('ohe',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['impute_categorical__HomePlanet',
                                                   'impute_categorical__CryoSleep',
                                                   'impute_categorical__VIP']),
                                                 ('or', OrdinalEncoder(),
                                                  ['impute_categorical__Destination'])])),
                ('model', HistGradientBoostingClassifier())])

In [59]:
X_train_ = pipe[:2].fit_transform(X_train)

In [60]:
X_train_.head(3)

,fu_age__fe_age_12__impute_numerical__Age,fu_age__fe_age_18__impute_numerical__Age,fu_age__fe_age_pass__impute_numerical__Age,pass__impute_numerical__RoomService,pass__impute_numerical__FoodCourt,pass__impute_numerical__ShoppingMall,pass__impute_numerical__Spa,pass__impute_numerical__VRDeck,fe_cabin__impute_cabin__Cabin_A,fe_cabin__impute_cabin__Cabin_B,...,fe_cabin__impute_cabin__Cabin_NA,fe_cabin__impute_cabin__Cabin_T,ohe__impute_categorical__HomePlanet_Earth,ohe__impute_categorical__HomePlanet_Europa,ohe__impute_categorical__HomePlanet_Mars,ohe__impute_categorical__CryoSleep_False,ohe__impute_categorical__CryoSleep_True,ohe__impute_categorical__VIP_False,ohe__impute_categorical__VIP_True,or__impute_categorical__Destination
PassengerId,,,,,,,,,,,,,,,,,,,,,
0001_01,1.0,1.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0
0002_01,1.0,1.0,24.0,109.0,9.0,25.0,549.0,44.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0
0003_01,1.0,1.0,58.0,43.0,3576.0,0.0,6715.0,49.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2.0


In [61]:
y.value_counts()

Transported
True     4378
False    4315
Name: count, dtype: int64

In [62]:
_ = cross_val_score(estimator=pipe, X=X_train, y=y, cv=10)

In [63]:
# Benchmark -                     0.7234581961033293
# New Model con OHE -             0.7155214740155814
# New Model con 4 depth -         0.764529582159438
# Model with 4 depth y CabinFe -  0.7709713900242054
# HistGradient default params y - 0.7998441860772721
# HistGradient default params y - 0.8028344113328835  # Leaderboard - 1359
# default pipe + OE de Destionation
np.mean(_)

0.8028344113328835

In [64]:
np.std(_)

0.01176739213356235

In [65]:
_

array([0.79425287, 0.7816092 , 0.79310345, 0.81357883, 0.79171461,
       0.80782509, 0.81357883, 0.81127733, 0.82048331, 0.8009206 ])

In [66]:
pipe.fit(X_train, y)

Pipeline(steps=[('impute',
                 ColumnTransformer(transformers=[('impute_numerical',
                                                  SimpleImputer(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x166636e80>),
                                                 ('impute_cabin',
                                                  SimpleImputer(fill_value='NA',
                                                                strategy='constant'),
                                                  ['Cabin']),
                                                 ('impute_categorical',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['HomePlanet', 'CryoSleep',
                                                   'Destinatio...
                                                  Pipeline(steps=[('extract_first_letter',
                                                                   CabinTransformer()),
                                                                  ('ohe_first_letter',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  ['impute_cabin__Cabin']),
                                                 ('ohe',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['impute_categorical__HomePlanet',
                                                   'impute_categorical__CryoSleep',
                                                   'impute_categorical__VIP']),
                                                 ('or', OrdinalEncoder(),
                                                  ['impute_categorical__Destination'])])),
                ('model', HistGradientBoostingClassifier())])

# Model Explainability

In [67]:
pipe

Pipeline(steps=[('impute',
                 ColumnTransformer(transformers=[('impute_numerical',
                                                  SimpleImputer(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x166636e80>),
                                                 ('impute_cabin',
                                                  SimpleImputer(fill_value='NA',
                                                                strategy='constant'),
                                                  ['Cabin']),
                                                 ('impute_categorical',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['HomePlanet', 'CryoSleep',
                                                   'Destinatio...
                                                  Pipeline(steps=[('extract_first_letter',
                                                                   CabinTransformer()),
                                                                  ('ohe_first_letter',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  ['impute_cabin__Cabin']),
                                                 ('ohe',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['impute_categorical__HomePlanet',
                                                   'impute_categorical__CryoSleep',
                                                   'impute_categorical__VIP']),
                                                 ('or', OrdinalEncoder(),
                                                  ['impute_categorical__Destination'])])),
                ('model', HistGradientBoostingClassifier())])

In [68]:
pipe[-1]

HistGradientBoostingClassifier()

In [69]:
if isinstance(pipe[-1], DecisionTreeClassifier):
    
    fig = plt.figure(figsize = (15, 20))
    ax = fig.subplots()

    plot_tree(
        decision_tree = pipe[-1],
        feature_names = X_train_.columns,
        ax = ax,
        fontsize = 8
    )

    plt.tight_layout()

# Submission a Kaggle

In [70]:
y_pred = pipe.predict(X_test)

In [71]:
timestamp = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
timestamp

'2024_03_30_19_46_37'

In [72]:
X_test_copy = X_test.copy(deep = True)

In [73]:
X_test_copy["Transported"] = list(map(bool, y_pred))

In [74]:
(
    X_test_copy
    .reset_index()
    [["PassengerId", "Transported"]]
    .to_csv(f"submission_{timestamp}.csv", index=False)
)

In [75]:
0.82043 - 0.79307

0.02735999999999994